In [31]:
!pip install pystac xarray pandas pystac-client

2385.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [1]:
import pystac
import xarray as xr
import pandas as pd
from datetime import datetime, date
from pystac_client import Client


In [15]:
# STAC API root URL
URL = 'https://catalog.dive.edito.eu'
# URL = 'https://s3.waw3-1.cloudferro.com/emodnet/bio_oracle/stac/catalog.json'
# custom headers
headers = []

cat = Client.open(URL, headers=headers)
cat

<Client id=catalogs>

Here we look for all the collections with chlorphyll in the id.  Here we can find items inside that have chlorophyll data.  We go through each item in one collection and print the assets.


In [16]:
collections = list(cat.get_all_collections())
print(f'Found {len(collections)} collections')
for collection in collections:
    if 'chlorophyll' in collection.id:
        print(collection.id)
        for item in collection.get_all_items():
            print(item.id)
            print(item.assets)
            break

/home/samwork/miniforge3/envs/tools/lib/python3.9/site-packages/pystac_client/client.py:440: FallbackToPystac: Falling back to pystac. This might be slow.
  self._warn_about_fallback("COLLECTIONS", "FEATURES")


Found 9690 collections
emodnet-deepest_values_of_water_body_chlorophyll_a
6a886250-2dff-53e6-8bcb-a99d6ff851b6
{'native_asset': <Asset href=https://s3.waw3-1.cloudferro.com/emodnet/emodnet_native/emodnet_chemistry/water_body_chlorophyll_a/water_body_chlorophyll_a_masked_using_relative_error_threshold_0.3_northeast_atlantic_ocean/Water_body_chlorophyll-a.4Danl.nc>, 'metadata_csw': <Asset href=https://emodnet.ec.europa.eu/geonetwork/emodnet/eng/csw?request=GetRecordById&service=CSW&version=2.0.2&elementSetName=full&id=a3461fb1-d209-440e-a49f-7acff7731395>, 'metadata_xml': <Asset href=https://emodnet.ec.europa.eu/geonetwork/srv/api/records/a3461fb1-d209-440e-a49f-7acff7731395/formatters/xml>, 'Zarr': <Asset href=https://s3.waw3-1.cloudferro.com/emodnet/emodnet_arco/emodnet_chemistry/water_body_chlorophyll_a/deepest_values_of_water_body_chlorophyll_a_northeast_atlantic_ocean/Water_body_chlorophyll-a.4Danl.zarr>, 'wms': <Asset href=https://ec.oceanbrowser.net/emodnet/Python/web/wms?service=

Here we will look more specifically for STAC items that from collections that have data with dissolved_oxygen, habitat, and temperature.  We will save each item in a dataframe

In [23]:
import pandas as pd

collections = list(cat.get_all_collections())
all_items = []
collection_selection = ['oxygen', 'habitat', 'elevation', 'temperature']
for collection in collections:
    if 'oxygen' in collection.id or 'habitat' in collection.id or 'temperature' in collection.id:
        collection_items = collection.get_all_items()
        try:
            for item in collection_items:
                # Append item information to the list
                all_items.append({  'Collection ID': collection.id, 
                                    'Item ID': item.id, 
                                    'item_starttime': item.properties['start_datetime'],
                                    'item_endtime': item.properties['end_datetime'], 
                                    'Assets': item.assets})
        except Exception as e:
            print(e)
            print(f'Error with {collection.id}')
            continue
oxygen_habitat_temperature_items_df = pd.DataFrame(all_items)


/home/samwork/miniforge3/envs/tools/lib/python3.9/site-packages/pystac_client/client.py:440: FallbackToPystac: Falling back to pystac. This might be slow.
  self._warn_about_fallback("COLLECTIONS", "FEATURES")


'list' object has no attribute 'items'
Error with climate_forecast-sea_water_temperature
'list' object has no attribute 'items'
Error with climate_forecast-sea_water_temperature


In [29]:
oxygen_habitat_temperature_items_df = pd.DataFrame(all_items)
oxygen_habitat_temperature_items_df

We can now filter the items based on the start and end time, look for items with start time after 2000 and end time before 2020

In [30]:
time_items_df = pd.DataFrame(all_items)

time_items_df['item_starttime'] = pd.to_datetime(time_items_df['item_starttime'])
time_items_df['item_endtime'] = pd.to_datetime(time_items_df['item_endtime'])

time_items_df = time_items_df.sort_values(by='item_starttime')
time_items_df

new_time_items_df = time_items_df[(time_items_df['item_starttime'] > '2000-01-01') & (time_items_df['item_endtime'] < '2030-12-31')]

new_time_items_df   

,Collection ID,Item ID,item_starttime,item_endtime,Assets
1001,climate_forecast-fractional_saturation_of_oxyg...,f8ee5c99-4e1f-52d1-97f3-20bc5a7f9d9b,2001-01-04 16:47:52+00:00,2023-06-30 23:59:11+00:00,{'arco-time-series': <Asset href=https://s3.wa...
58,climate_forecast-fractional_saturation_of_oxyg...,98c8594e-d438-5b9b-a02a-e911480e3c1a,2001-01-04 16:47:52+00:00,2023-06-30 23:59:11+00:00,{'arco-geo-series': <Asset href=https://s3.waw...
57,climate_forecast-fractional_saturation_of_oxyg...,f8ee5c99-4e1f-52d1-97f3-20bc5a7f9d9b,2001-01-04 16:47:52+00:00,2023-06-30 23:59:11+00:00,{'arco-time-series': <Asset href=https://s3.wa...
963,climate_forecast-air_temperature,650d1f3a-88a6-50c6-ac6f-a509030a4679,2001-01-04 16:47:52+00:00,2023-06-30 23:59:11+00:00,{'arco-time-series': <Asset href=https://s3.wa...
14,climate_forecast-air_temperature,650d1f3a-88a6-50c6-ac6f-a509030a4679,2001-01-04 16:47:52+00:00,2023-06-30 23:59:11+00:00,{'arco-time-series': <Asset href=https://s3.wa...
...,...,...,...,...,...
1401,climate_forecast-sea_water_potential_temperature,a1ea3377-0f77-5d23-9bfa-eea49071edfa,2024-09-11 00:00:00+00:00,2024-09-29 23:00:00+00:00,{'wmts': <Asset href=https://wmts.marine.coper...
459,climate_forecast-sea_water_potential_temperature,2f783bc6-a565-53c5-a5ff-ea5fdc4df2b1,2024-09-11 00:00:00+00:00,2024-09-29 23:00:00+00:00,{'arco-geo-series': <Asset href=https://s3.waw...
460,climate_forecast-sea_water_potential_temperature,a1ea3377-0f77-5d23-9bfa-eea49071edfa,2024-09-11 00:00:00+00:00,2024-09-29 23:00:00+00:00,{'wmts': <Asset href=https://wmts.marine.coper...
458,climate_forecast-sea_water_potential_temperature,d07e6c1a-7cef-5bae-8981-05a319aec8cb,2024-09-11 00:00:00+00:00,2024-09-29 23:00:00+00:00,{'arco-time-series': <Asset href=https://s3.wa...


Lastly we will look specifically for Analysis Ready Cloud Optimized assets in each of the items, specifically Zarr and Parquet datasets.  These will feature variables that have dissolved_oxygen, habitat, or temperature

In [31]:
all_items_assets = []
           # Now filter the assets
all_items_assets = []
for _, row in time_items_df.iterrows():
    collection_id = row['Collection ID']
    item_id = row['Item ID']
    data_starttime = row['item_starttime']
    data_endtime = row['item_endtime']
    assets = row['Assets']
    
    for asset_key, asset in assets.items():
        if asset.href.endswith('.zarr') or asset.href.endswith('.zarr/') or asset.href.endswith('.parquet'):
            # Append asset information to the list
            all_items_assets.append({'Collection ID': collection_id, 'Item ID': item_id, 'Data Start': data_starttime, 'Data End': data_endtime, 'Asset Key': asset_key, 'Asset Href': asset.href})

# Create a DataFrame for assets
assets_df = pd.DataFrame(all_items_assets)
assets_df

,Collection ID,Item ID,Data Start,Data End,Asset Key,Asset Href
0,emodnet-water_body_dissolved_oxygen_concentrat...,c93d0332-ea7b-55e8-9e0a-1fa9b718500a,1899-12-30 13:27:00+00:00,2022-02-08 09:15:00+00:00,Zarr,https://s3.waw3-1.cloudferro.com/emodnet/emodn...
1,emodnet-water_body_dissolved_oxygen_concentrat...,c93d0332-ea7b-55e8-9e0a-1fa9b718500a,1899-12-30 13:27:00+00:00,2022-02-08 09:15:00+00:00,Zarr-datalab-data-explorer,https://datalab.dive.edito.eu/data-explorer?so...
2,emodnet-water_body_dissolved_oxygen_concentrat...,c93d0332-ea7b-55e8-9e0a-1fa9b718500a,1899-12-30 13:27:00+00:00,2022-02-08 09:15:00+00:00,Zarr,https://s3.waw3-1.cloudferro.com/emodnet/emodn...
3,emodnet-water_body_dissolved_oxygen_concentrat...,c93d0332-ea7b-55e8-9e0a-1fa9b718500a,1899-12-30 13:27:00+00:00,2022-02-08 09:15:00+00:00,Zarr-datalab-data-explorer,https://datalab.dive.edito.eu/data-explorer?so...
4,emodnet-water_body_dissolved_oxygen_concentrat...,b95a522b-5734-54b6-a254-d57a27047fac,1960-01-21 15:00:00+00:00,2020-12-31 11:32:00+00:00,Zarr,https://s3.waw3-1.cloudferro.com/emodnet/emodn...
...,...,...,...,...,...,...
2059,climate_forecast-sea_water_potential_temperature,2f783bc6-a565-53c5-a5ff-ea5fdc4df2b1,2024-09-11 00:00:00+00:00,2024-09-29 23:00:00+00:00,arco-geo-series-datalab-data-explorer,https://datalab.dive.edito.eu/data-explorer?so...
2060,climate_forecast-sea_water_potential_temperature,d07e6c1a-7cef-5bae-8981-05a319aec8cb,2024-09-11 00:00:00+00:00,2024-09-29 23:00:00+00:00,arco-time-series,https://s3.waw3-1.cloudferro.com/mdl-arco-geo-...
2061,climate_forecast-sea_water_potential_temperature,d07e6c1a-7cef-5bae-8981-05a319aec8cb,2024-09-11 00:00:00+00:00,2024-09-29 23:00:00+00:00,arco-time-series-datalab-data-explorer,https://datalab.dive.edito.eu/data-explorer?so...
2062,climate_forecast-sea_water_potential_temperature,2f783bc6-a565-53c5-a5ff-ea5fdc4df2b1,2024-09-11 00:00:00+00:00,2024-09-29 23:00:00+00:00,arco-geo-series,https://s3.waw3-1.cloudferro.com/mdl-arco-time...


Now we can specifically find what kind of data we want. Dissolved oxygen, eunis_seabed_habitat_class, etc.  And with the link the the assets we can subset parts of large datasets.  

Find an array with sea water potential temperature over the atlantic ocean, between 2010 and 2020.  Any array with seabed habitats.  And Dissolved oxygen over the atlantic ocean, between 2010 and 2020.

In [ ]:
oxygen_assets = assets_df[assets_df['Collection ID'].str.contains('oxygen')]

for zarr_url in oxygen_assets['Asset Href']:
    print(zarr_url)
    ds = xr.open_zarr(zarr_url)
    print(ds)
    break